<a href="https://colab.research.google.com/github/Sartify/Gemma-3-Notebooks/blob/main/scripts/text_inference_gemma_without_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the required dependencies for using Hugging Face's Transformers library
# This specific version includes support for Google's Gemma 3 model
!pip install -q git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

# Upgrade and install bitsandbytes, a library for efficient model loading and quantization
!pip install -U -q bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 769.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.9 MB/s

In [2]:
# Import necessary libraries

# Hugging Face's Transformers library for working with Gemma 3 model
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

# PIL (Python Imaging Library) for handling image processing
from PIL import Image

# Requests library to fetch images from URLs
import requests

# PyTorch for tensor computations and model execution
import torch

In [3]:
# Define the model ID - you can change this to any available Gemma 3 model variant
model_id = "google/gemma-3-4b-it"

# Load the Gemma 3 model for conditional generation
# device_map="auto" automatically assigns the model to an available GPU or CPU
model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()  # Set the model to evaluation mode (inference mode)

# Load the corresponding processor for the selected model
# The processor is responsible for preprocessing text and images before feeding them into the model
processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [4]:
# Define the conversation messages for the model
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]  # System message to set model behavior
    },
    {
        "role": "user",
        "content": [
            # Uncomment the next line to include an image for multimodal input
            # {"type": "image", "image": ""},
            {"type": "text", "text": "solve the following simultaneous equations 2x+3y=12,  4x−y=5 tafuta x na y "}
            # User input requesting the solution to the system of equations
        ]
    }
]

# Convert the messages into a format suitable for the model
inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,  # Adds a generation prompt for response generation
    tokenize=True,  # Tokenizes the input text
    return_dict=True,  # Returns output as a dictionary
    return_tensors="pt"  # Returns PyTorch tensors for processing
).to(model.device, dtype=torch.bfloat16)  # Move to the correct device and data type for efficiency

# Capture the input length to ensure proper decoding later
input_len = inputs["input_ids"].shape[-1]

# Generate model output without storing gradients for inference efficiency
with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1024, do_sample=False)
    # Generate response with a max token limit, without randomness (deterministic output)

    generation = generation[0][input_len:]  # Extract only the generated portion of the output

# Decode the generated output into human-readable text
decoded = processor.decode(generation, skip_special_tokens=True)

# Print the final response
print(decoded)

Okay, let's solve the simultaneous equations:

**Equation 1:** 2x + 3y = 12
**Equation 2:** 4x - y = 5

Here's how we can solve this using the substitution method:

1. **Solve Equation 2 for y:**
   4x - y = 5
   -y = 5 - 4x
   y = 4x - 5

2. **Substitute this value of y into Equation 1:**
   2x + 3(4x - 5) = 12

3. **Simplify and solve for x:**
   2x + 12x - 15 = 12
   14x = 27
   x = 27/14

4. **Substitute the value of x back into the equation y = 4x - 5 to find y:**
   y = 4 * (27/14) - 5
   y = 108/14 - 5
   y = 54/7 - 35/7
   y = 19/7

**Therefore:**

*   x = 27/14
*   y = 19/7

**Let me know if you'd like me to show you how to solve it using another method, like elimination!**
